In [1]:
%load_ext autoreload
%autoreload 2
from datamodules import FMDataset, fm_collate, CFMDataset, SCFMDataset, cfm_collate
from arch import *
from flow_utils import compute_conditional_flow

import scvi
import torch
import numpy as np
import pytorch_lightning as pl
import os

import scanpy as sc

from torchcfm.conditional_flow_matching import *
import scanpy as sc
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

In [2]:
# load some data
adata = sc.read_h5ad('../../../dlesman/datasets/cellot/kaggle_HVG_cellot.h5ad')

# here we set up the train/eval and control/pert sets
# set the idx of the controls
control_idx = adata.obs['sm_name'] == 'Dimethyl Sulfoxide'
# set the idx of the perts (currently just "all not control")
pert_idx = adata.obs['sm_name'] != 'Dimethyl Sulfoxide'
# set the hold out cell-type/pert
eval_cell_idx = adata.obs.cell_type == 'B cells'
eval_pert_idx = adata.obs['sm_name'] == 'Belinostat'
eval_idx = eval_cell_idx & eval_pert_idx

In [3]:
# here we set up our embeddings for cfm
# this is just so everything lives in obsm for the for loop below
adata.obsm["X"] = adata.X

# this is an example of how we can embed something using just the train idxs
# and then run fm on that embedding
# embedder = PCA(n_components=30).fit(adata.X[(control_idx | pert_idx) & ~eval_idx])
# adata.obsm["X_pca"] = embedder.transform(adata.X)

In [4]:
# here we set up the perts
import pandas as pd
perts = pd.get_dummies(adata.obs['sm_name']).values.astype(float)
pert_ids = perts.argmax(axis=1)
# this is the "identity featurization"; we can swap this matrix for
# any latent representation of perturbations we want but this is 
# a non-parametric featurization for right now
pert_mat = np.eye(pert_ids.max() + 1).astype('float32')

In [5]:
cell_types = pd.get_dummies(adata.obs.cell_type).values.argmax(axis=1)

In [11]:
X = adata.obsm['X']

# set train and eval split
control_train = X[control_idx & ~eval_idx]
pert_train = X[pert_idx & ~eval_idx]
pert_ids_train =  pert_ids[pert_idx & ~eval_idx]
control_cell_types = cell_types[control_idx & ~eval_idx]
pert_cell_types = cell_types[pert_idx & ~eval_idx]

control_eval = X[control_idx & eval_cell_idx]
pert_eval = X[eval_idx]
pert_ids_eval = pert_ids[eval_idx]

batch_size = 32

In [12]:
dset = CFMDataset(
    control_train, pert_train, 
    pert_ids_train, pert_mat, 
)
dl = torch.utils.data.DataLoader(dset, batch_size=batch_size, collate_fn=cfm_collate)

In [15]:
next(iter(dl))

(tensor([0.7672, 0.8924, 0.2026, 0.6702, 0.2146, 0.8444, 0.9188, 0.4461, 0.7909,
         0.2075, 0.9298, 0.1816, 0.9812, 0.1775, 0.9585, 0.3044, 0.9039, 0.4690,
         0.0286, 0.3877, 0.5945, 0.5882, 0.2413, 0.8968, 0.6402, 0.0442, 0.9173,
         0.8214, 0.6086, 0.5795, 0.2541, 0.6718]),
 tensor([[-1.3740e-01, -1.1277e-01, -7.7703e-02,  ...,  1.0712e-01,
           2.0328e-01, -1.0614e-03],
         [-1.9844e-01,  1.0360e-02, -8.4946e-02,  ...,  8.0624e-01,
           4.6086e-02,  4.5000e-02],
         [-8.2162e-02, -7.2899e-02, -1.3968e-01,  ...,  7.4736e-02,
           2.4865e-02,  4.9716e-02],
         ...,
         [-4.9889e-02, -1.4776e-01,  1.4923e-02,  ..., -2.3738e-02,
           4.6483e-02,  5.3594e-02],
         [-1.6304e-01,  1.0963e+00,  2.8482e-01,  ...,  1.2157e-02,
          -9.0748e-03, -1.9810e-01],
         [ 3.0687e-02,  1.5075e+00, -2.5238e-03,  ..., -7.1484e-02,
          -8.4791e-02, -7.8402e-03]]),
 tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,

In [ ]:
dset = SCFMDataset(
    control_train, pert_train, 
    pert_ids_train, pert_mat, 
    control_cell_types, pert_cell_types,
    batch_size=batch_size
)
dl = torch.utils.data.DataLoader(dset, batch_size=batch_size, collate_fn=cfm_collate)

In [6]:
for embedding in ["X"]: # if we wanted to run in the PCA "latent" as well we would use ["X", "X_pca"]
    print(embedding)
    # set X to the (latent) counts
    X = adata.obsm[embedding]
    
    # set train and eval split
    control_train = X[control_idx & ~eval_idx]
    pert_train = X[pert_idx & ~eval_idx]
    pert_ids_train =  pert_ids[pert_idx & ~eval_idx]
    control_cell_types = cell_types[control_idx & ~eval_idx]
    pert_cell_types = cell_types[pert_idx & ~eval_idx]
    
    control_eval = X[control_idx & eval_cell_idx]
    pert_eval = X[eval_idx]
    pert_ids_eval = pert_ids[eval_idx]

    # set up data processing for cfm
    batch_size = 32
    dset = CFMDataset(
        control_train, pert_train, 
        pert_ids_train, pert_mat, 
        control_cell_types, pert_cell_types,
        batch_size=batch_size
    )
    dl = torch.utils.data.DataLoader(dset, batch_size=batch_size, collate_fn=cfm_collate)
    
    # Train the model
    trainer = pl.Trainer(
        gpus=1,  # Specify the number of GPUs to use
        max_epochs=1_000,  # Specify the maximum number of training epochs
    )
    model = CMLP(feat_dim=X.shape[1], cond_dim=pert_mat.shape[1], time_varying=True)
    trainer.fit(model, dl)

    # Save the predicted perturbation
    torch.cuda.empty_cache()
    traj = compute_conditional_flow(model, control_eval, pert_ids_eval, pert_mat)

    np.savez(
        f"kaggle.{embedding}.cmlp.identity.npz", 
        pred_pert=traj[-1, :, :], true_pert=pert_eval, control=control_eval, traj=traj
    )
    np.savez(
        f"kaggle.{embedding}.cmlp.identity.strat.npz", 
        pred_pert=traj[-1, :, :], true_pert=pert_eval, control=control_eval
    )

X


/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/ ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[0.11802492 0.17374211 0.19115699 0.43906356 0.053642   0.02437041]


/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | combo_net | Sequential | 266 K 
1 | cond_net  | Sequential | 17.9 K
-----------------------------------------
284 K     Trainable params
0         Non-trainable params
284 K     Total params
1.137     Total estimated model params size (MB)
/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.


Training: 0it [00:00, ?it/s]

/om2/user/njwfish/anaconda/envs/dsbm/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: Detected KeyboardInterrupt, attempting graceful shutdown...


KeyboardInterrupt: 

In [ ]:
embedder = PCA(n_components=2).fit(adata.X[((control_idx & eval_cell_idx) | eval_idx)])

source_embed = embedder.transform(X[control_idx & eval_cell_idx])
target_embed = embedder.transform(X[eval_idx])
traj_embed = np.stack([embedder.transform(traj_i) for traj_i in traj])


fig = plt.figure(figsize=(10, 4))
axs = fig.subplots(1, 2, sharey=True, sharex=True)
k = 10_000
axs[0].scatter(source_embed[:k, 0], source_embed[:k, 1], alpha=0.3, s=1, label="source", color='blue')
axs[0].scatter(target_embed[:k, 0], target_embed[:k, 1], alpha=0.3, s=1, label="target", color='green')
axs[0].scatter(traj_embed[-1, :k, 0], traj_embed[-1, :k, 1], alpha=0.3, s=1, label="predicted", color='red')
axs[0].legend()

k = 300
axs[1].scatter(source_embed[:k, 0], source_embed[:k, 1], alpha=0.7, s=2, label="source", color='blue')
axs[1].scatter(traj_embed[-1, :k, 0], traj_embed[-1, :k, 1], alpha=0.7, s=2, label="predicted", color='red')

for i in range(k):
    axs[1].plot(traj_embed[:, i, 0].T, traj_embed[:, i, 1].T, color="purple", alpha=0.1)
plt.show()

In [ ]:
for embedding in ["X"]: # if we wanted to run in the PCA "latent" as well we would use ["X", "X_pca"]
    print(embedding)
    # set X to the (latent) counts
    X = adata.obsm[embedding]
    
    # set train and eval split
    control_train = X[control_idx & ~eval_idx]
    pert_train = X[pert_idx & ~eval_idx]
    pert_ids_train =  pert_ids[pert_idx & ~eval_idx]
    control_cell_types = cell_types[control_idx & ~eval_idx]
    pert_cell_types = cell_types[pert_idx & ~eval_idx]
    
    control_eval = X[control_idx & eval_cell_idx]
    pert_eval = X[eval_idx]
    pert_ids_eval = pert_ids[eval_idx]

    # set up data processing for cfm
    batch_size = 32
    dset = SCFMDataset(
        control_train, pert_train, 
        pert_ids_train, pert_mat, 
        control_cell_types, pert_cell_types,
        batch_size=batch_size
    )
    dl = torch.utils.data.DataLoader(dset, batch_size=32, collate_fn=cfm_collate)
    
    # Train the model
    trainer = pl.Trainer(
        gpus=1,  # Specify the number of GPUs to use
        max_epochs=1_000,  # Specify the maximum number of training epochs
    )
    model = CMHA(feat_dim=X.shape[1], cond_dim=pert_mat.shape[1], time_varying=True)
    trainer.fit(model, dl)

    # Save the predicted perturbation
    torch.cuda.empty_cache()
    traj = compute_conditional_flow(model, control_eval, pert_ids_eval, pert_mat)

    np.savez(
        f"kaggle.{embedding}.cmha.identity.npz", 
        pred_pert=traj[-1, :, :], true_pert=pert_eval, control=control_eval, traj=traj
    )
    
    np.savez(
        f"kaggle.{embedding}.cmha.identity.strat.npz", 
        pred_pert=traj[-1, :, :], true_pert=pert_eval, control=control_eval
    )

In [ ]:
embedder = PCA(n_components=2).fit(adata.X[((control_idx & eval_cell_idx) | eval_idx)])

source_embed = embedder.transform(X[control_idx & eval_cell_idx])
target_embed = embedder.transform(X[eval_idx])
traj_embed = np.stack([embedder.transform(traj_i) for traj_i in traj])


fig = plt.figure(figsize=(10, 4))
axs = fig.subplots(1, 2, sharey=True, sharex=True)
k = 10_000
axs[0].scatter(source_embed[:k, 0], source_embed[:k, 1], alpha=0.3, s=1, label="source", color='blue')
axs[0].scatter(target_embed[:k, 0], target_embed[:k, 1], alpha=0.3, s=1, label="target", color='green')
axs[0].scatter(traj_embed[-1, :k, 0], traj_embed[-1, :k, 1], alpha=0.3, s=1, label="predicted", color='red')
axs[0].legend()

k = 300
axs[1].scatter(source_embed[:k, 0], source_embed[:k, 1], alpha=0.7, s=2, label="source", color='blue')
axs[1].scatter(traj_embed[-1, :k, 0], traj_embed[-1, :k, 1], alpha=0.7, s=2, label="predicted", color='red')

for i in range(k):
    axs[1].plot(traj_embed[:, i, 0].T, traj_embed[:, i, 1].T, color="purple", alpha=0.1)
plt.show()

In [24]:
print("test")

test
